In [1]:
import pandas as pd
import pymongo
import os
import sys
sys.path.append('../')
from MatchingModule import recommend as rc

In [2]:
df_job = rc.load_job_data()
df_user = rc.load_user_data()

In [3]:
df_user

,_id,experience,user_vector
0,ACoAACGoyx8BN_W_i04AdC-kWSjlP3laMt2-ATo,"[FPT Corporation, Talent Acquisition Leader, S...","[en world group, fpt corporation, rpo recruite..."
1,trucnguyen1112,"[People Data Analyst, Real Estate Sales Execut...","[microsoft excel, machine learning, decision-m..."
2,ACoAAA4hY4YBg3-UH5yt-qQmQoySgGg5dVL4vX4,"[Expat Local, Co-Founder, Co-Founder, Founder,...","[pvep ovs., expat local, information technolog..."
3,ACoAACdYwZwBgXSpz35E99EYMaUmdt-MtHMYe0Y,[],"[truòng dai hoc da lat, melinda french gates, ..."
4,results,"[Sr. Technical Project Manager, Program Manage...","[bachelors, management, masters science, bache..."
...,...,...,...
6450,tqhernandez,"[Staff Scientist, Associate, Jr Data Scientist...","[bachelor science, microsoft excel, people ski..."
6451,kevin-hew-19349513,"[Development Team Lead, Gamesys, Development T...","[bachelor commerce, team management, managemen..."
6452,ducngtrung,"[Tek Experts, Technical Support Engineer level...","[telecommunication engineering, vietnam nation..."
6453,dangnh0611,"[AI Engineer, Machine Learning Engineer, Machi...","[information technology, engineers, degree]"


In [4]:
df_job

,job_id,jobTitle,skills
0,2975673294,Data Engineer in Business Intelligence Team (B...,"[creative approaches implementation, sources, ..."
1,2993666500,"Business Intelligence Intern, Kobiton","[every quarter, attitude, flexible time, platf..."
2,2975674182,"Business Intelligence Developer, Product Team ...","[creative approaches implementation, sources, ..."
3,2993740820,"Business Intelligence (Edtech Company, Upto 10...","[1 2 times, paid, travel, engineer, business, ..."
4,2945707463,Senior Business Intelligence Analyst (0824),"[sources, align stakeholders, deep information..."
...,...,...,...
1064,2920919416,Senior Automation Tester,"[expereince testng, responsibilities, europe, ..."
1065,2920916624,Senior Automation Tester,"[expereince testng, responsibilities, europe, ..."
1066,2920917626,Senior Automation Tester,"[expereince testng, responsibilities, europe, ..."
1067,2690306027,QC Manager (Tester),"[japanese class, documents, information techno..."


<h2>Evaluate by Calculate F1 mean score
    </h2>

In [ ]:
# Get max similarity score
max = 0
score = 0
for i in range(0,6455):
    score = rc.max_similarity(df_user.iloc[i]['experience'],df_user.iloc[i]['user_vector'],df_job)
    # print(df.iloc[0]['similarity'])
    if max < score:
        max = score
    print("Current Max: ",max,' counter: ',i)
print("FIANAL MAX: ",max)

<h3>Define threshold score</h3>

In [14]:
max = 11.979999999999999
max_score = max
# max_score
# for i in range (0,5):
# threshold_list = [0.1,0.125,0.15,0.175,0.2]
threshold_list = [x*0.05 for x in range(1,10)]
# threshold = threshold_list[0]
threshold_list 

[0.05,
 0.1,
 0.15000000000000002,
 0.2,
 0.25,
 0.30000000000000004,
 0.35000000000000003,
 0.4,
 0.45]

<h3>Get 10 users to test threshold</h3>

In [15]:
# Get 10 random users
from random import randrange
test_user_index = []
for i in range(0,5):
    t = randrange(0,5000)
    test_user_index.append(t)

print(test_user_index)

[1263, 951, 1000, 1243, 4137]


<h3>Calculate Precision & Recall & F1 score for each user</h3>

In [21]:
# Calculate Precision, F1 of single user recommend
# TP - True Positive = Number of item >= threshold
# FP - False Positve = Number of item removed from Data frame
# TN - True Negative = Number of matches with score which is equal to zero is considered as true negative
# FN - False Negative =  0 ---> recall = 1

def single_user(index,threshold):
    df_recs = rc.recommend_top_N(df_user.iloc[index]['experience'],df_user.iloc[index]['user_vector'],df_job,N=20)
    # df_recs = df_recs[df_recs['similarity']>=0]
    TP = len(df_recs[df_recs['similarity']>=threshold])
    FP = len(df_recs) - len(df_recs[df_recs['similarity']>=threshold])
    FN = 0
    recall = 1
    precision = TP/(TP+FP)
    f1 =  2*precision*recall/(precision+recall)
    return precision,f1

dict_list = []

for user in test_user_index:
    print("USER: ",user)
    temp_list = []
    for item in threshold_list:
        x,y = single_user(user,item*max)
        dic = {'threshold':item,'precision':x,'f1':y}
        temp_list.append(dic)
    
    dict_list.append({'user':user,'detail':temp_list})
        # print("Threshold: ",item,x,y)
    # print(threshold_list[1]*max)

USER:  1263
USER:  951
USER:  1000
USER:  1243
USER:  4137


In [22]:
dict_list

[{'user': 1263,
  'detail': [{'threshold': 0.05, 'precision': 1.0, 'f1': 1.0},
   {'threshold': 0.1, 'precision': 1.0, 'f1': 1.0},
   {'threshold': 0.15000000000000002,
    'precision': 0.35,
    'f1': 0.5185185185185185},
   {'threshold': 0.2, 'precision': 0.0, 'f1': 0.0},
   {'threshold': 0.25, 'precision': 0.0, 'f1': 0.0},
   {'threshold': 0.30000000000000004, 'precision': 0.0, 'f1': 0.0},
   {'threshold': 0.35000000000000003, 'precision': 0.0, 'f1': 0.0},
   {'threshold': 0.4, 'precision': 0.0, 'f1': 0.0},
   {'threshold': 0.45, 'precision': 0.0, 'f1': 0.0}]},
 {'user': 951,
  'detail': [{'threshold': 0.05, 'precision': 1.0, 'f1': 1.0},
   {'threshold': 0.1, 'precision': 1.0, 'f1': 1.0},
   {'threshold': 0.15000000000000002, 'precision': 1.0, 'f1': 1.0},
   {'threshold': 0.2, 'precision': 1.0, 'f1': 1.0},
   {'threshold': 0.25, 'precision': 0.75, 'f1': 0.8571428571428571},
   {'threshold': 0.30000000000000004,
    'precision': 0.55,
    'f1': 0.7096774193548387},
   {'threshold': 0

In [25]:
# Avarage precision each user

for item in dict_list:
    user = item['user']
    details = item['detail']
    sum_precision = 0
    sum_f1 = 0
    for s in details:
        sum_precision += s['precision']
        sum_f1 += s['f1']
    print("USER: ",item['user'])
    print("AVG precision: ",sum_precision/len(details))
    print("AVG f1: ",sum_f1/len(details))

    # avg = 

USER:  1263
AVG precision:  0.2611111111111111
AVG f1:  0.27983539094650206
USER:  951
AVG precision:  0.6
AVG f1:  0.6396996074415429
USER:  1000
AVG precision:  0.6222222222222222
AVG f1:  0.6642616642616642
USER:  1243
AVG precision:  0.5444444444444444
AVG f1:  0.6022122142307663
USER:  4137
AVG precision:  0.35555555555555557
AVG f1:  0.3703703703703704


In [32]:
# Average Precision & f1 by threshold
for threshold in threshold_list:
    print("THRESHOLD: ",threshold)
    sum_precision = 0
    sum_f1 = 0
    for item in dict_list:
        detail = item['detail']
        # print("DETAIL: ",detail)
        for x in detail:
            if( x['threshold'] == threshold):
                print("X: ",x['precision'])
                sum_precision += i['precision']
                sum_f1 += i['f1']
    print("Threshold: ",threshold)
    print("AVG precision: ",sum_precision/len(test_user_index))
    print("AVG f1: ",sum_f1/len(test_user_index))
    
    #     for u in range(len(dict_list['detail']):
    #         for i in u:
    #             if i['threshold'] == threshold:
    #                 sum_precision += i['precision']
    #                 sum_f1 += i['f1']
    

THRESHOLD:  0.05
X:  1.0
X:  1.0
X:  1.0
X:  1.0
X:  1.0
THRESHOLD:  0.1
X:  1.0
X:  1.0
X:  1.0
X:  1.0
X:  1.0
THRESHOLD:  0.15000000000000002
X:  0.35
X:  1.0
X:  1.0
X:  1.0
X:  1.0
THRESHOLD:  0.2
X:  0.0
X:  1.0
X:  1.0
X:  0.9
X:  0.2
THRESHOLD:  0.25
X:  0.0
X:  0.75
X:  0.75
X:  0.45
X:  0.0
THRESHOLD:  0.30000000000000004
X:  0.0
X:  0.55
X:  0.65
X:  0.35
X:  0.0
THRESHOLD:  0.35000000000000003
X:  0.0
X:  0.05
X:  0.2
X:  0.2
X:  0.0
THRESHOLD:  0.4
X:  0.0
X:  0.05
X:  0.0
X:  0.0
X:  0.0
THRESHOLD:  0.45
X:  0.0
X:  0.0
X:  0.0
X:  0.0
X:  0.0
